# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = run_engine(data)

pl    0.00236
dtype: float64
--------------------------------------
             DateTime_frmt          close_type       pl
101469 2021-01-04 18:39:22  simple_take_profit  0.00026
102377 2021-01-04 18:55:44  simple_take_profit  0.00022
103150 2021-01-04 19:07:03  simple_take_profit  0.00020
104686 2021-01-04 19:35:59  simple_take_profit  0.00011
105400 2021-01-04 19:50:55  simple_take_profit  0.00017
105865 2021-01-04 19:57:29  simple_take_profit  0.00025
106346 2021-01-04 20:03:06  simple_take_profit  0.00026
107726 2021-01-04 20:28:54         simple_stop -0.00030
108741 2021-01-04 20:48:57  simple_take_profit  0.00023
110218 2021-01-04 21:21:12  simple_take_profit  0.00031
110690 2021-01-04 21:40:37  simple_take_profit  0.00034
111449 2021-01-04 22:02:39  simple_take_profit  0.00039
111491 2021-01-04 22:06:19  simple_take_profit  0.00051
111647 2021-01-04 22:18:55  simple_take_profit  0.00048
112736 2021-01-04 23:29:33  simple_take_profit  0.00034


100%|███████████████████████████████| 113572/113572 [00:10<00:00, 10705.19it/s]


Wall time: 1min 6s


In [3]:
generate_result_report(data)

--------------------------------------
Run Complete : data/ema_df-(2021-2021)-(1-1)-(1-4).csv
--------------------------------------


--------------------------------------
Net PL
0.0024
--------------------------------------


--------------------------------------
Monthly PL
                pl
month_val         
1          0.00236
--------------------------------------


--------------------------------------
Daily PL
               pl
date_val         
3        -0.00104
4         0.00340
--------------------------------------


--------------------------------------
Count of transactions by Close Type
..................
                     pl
close_type             
simple_stop          54
simple_take_profit  106
--------------------------------------


--------------------------------------
Mean value of transactions by Close Type
..................
                          pl
close_type                  
simple_stop        -0.000452
simple_take_profit  0.000253
----------------

In [5]:
data['df']

,DateTime_frmt,month_val,date_val,hour_val,Bid,Ask,tick,lema,h,l,h_l_gap,h_lema,l_lema,long_open,long_close,short_open,short_close,close_type,pl
0,2021-01-03 23:03:51,1,3,23,1.22338,1.22351,1.223445,1.223498,1.224060,1.22327,0.000790,1.224060,1.223270,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-03 23:03:51,1,3,23,1.22338,1.22353,1.223455,1.223498,1.224060,1.22327,0.000790,1.224060,1.223270,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03 23:03:51,1,3,23,1.22338,1.22352,1.223450,1.223498,1.224060,1.22327,0.000790,1.224060,1.223270,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-03 23:03:51,1,3,23,1.22338,1.22351,1.223445,1.223498,1.224060,1.22327,0.000790,1.224060,1.223270,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-03 23:03:51,1,3,23,1.22338,1.22350,1.223440,1.223498,1.224060,1.22327,0.000790,1.224060,1.223270,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113567,2021-01-04 23:59:28,1,4,23,1.22524,1.22527,1.225255,1.225154,1.225345,1.22501,0.000335,1.225345,1.225010,NaN,NaN,NaN,NaN,NaN,NaN
113568,2021-01-04 23:59:28,1,4,23,1.22524,1.22529,1.225265,1.225154,1.225345,1.22501,0.000335,1.225345,1.225010,NaN,NaN,NaN,NaN,NaN,NaN
113569,2021-01-04 23:59:29,1,4,23,1.22525,1.22530,1.225275,1.225154,1.225345,1.22501,0.000335,1.225345,1.225010,NaN,NaN,NaN,NaN,NaN,NaN
113570,2021-01-04 23:59:44,1,4,23,1.22525,1.22529,1.225270,1.225155,1.225345,1.22501,0.000335,1.225345,1.225010,NaN,NaN,NaN,NaN,NaN,NaN


0              NaN
1         1.224060
2         1.224060
3         1.224060
4         1.224060
            ...   
113567    1.225345
113568    1.225345
113569    1.225345
113570    1.225345
113571    1.225345
Name: h, Length: 113572, dtype: float64

In [10]:
data["plot"] = True
data['reduce_plot'] = False
data['plot_type'] = 'file'
data['plot_start']  = '2021-01-06'
data['plot_stop']   = '2021-01-07'

In [11]:
plot_graph(data)